In [1]:
import pandas as pd
from datetime import timedelta
import numpy as np
from dateutil.relativedelta import *
from collections import defaultdict

### Import Data

In [2]:
# Get historical crypto market cap rank data
cryptoMarketCapRankDf = pd.read_csv('data\CryptoMarketCap.csv')
cryptoMarketCapRankDf['Date'] = pd.to_datetime(cryptoMarketCapRankDf['Date'])

In [3]:
# Get historical crypto price data
cryptoPriceDf = pd.read_csv('data\TradingViewCryptoPrice.csv', index_col=0)
cryptoPriceDf.index = pd.to_datetime(cryptoPriceDf.index)

### Formation Period

In [4]:
def marketCapCryptoSelection(cryptoMarketCapRankDf, cutoffDate, cutoffRank):
    # Get formation period sample crypto list
    marketCapCutoffDate = pd.to_datetime(cutoffDate) - timedelta(days=1)
    sampleCrypto = cryptoMarketCapRankDf[(cryptoMarketCapRankDf['Date'] == marketCapCutoffDate) & (cryptoMarketCapRankDf['Rank'] <= cutoffRank)]
    sampleCrypto = list(sampleCrypto['Symbol'])
    return sampleCrypto

In [5]:
def cryptoPriceCleaning(cryptoPriceDf, sampleCrypto, cutoffDate, lookback):
    # Fliter the crypto with formation period
    cutoffRowIdx = cryptoPriceDf.index.get_loc(cutoffDate)
    if cutoffRowIdx < lookback:
        # if there is not enough got the whole lookback period, just get all the availiable data
        sampleCryptoPrice = cryptoPriceDf.iloc[:cutoffRowIdx]
    else:
        sampleCryptoPrice = cryptoPriceDf.iloc[cutoffRowIdx-lookback:cutoffRowIdx]
    
    # Filter based on the availiablity of crpyto price
    sampleCrypto = set(sampleCrypto).intersection([x[7:-3] for x in sampleCryptoPrice.columns])

    # Data Cleaning
    sampleCryptoPrice = sampleCryptoPrice[["CRYPTO:" + x + "USD" for x in sampleCrypto]]
    sampleCryptoPrice = sampleCryptoPrice.ffill(axis=0)
    sampleCryptoPrice = sampleCryptoPrice.dropna(axis=1)

    print('Remaining number of crpyto: ', len(sampleCryptoPrice.columns))

    return sampleCryptoPrice

#### Distance Method

In [6]:
def distanceMethodSelection(sampleCryptoPrice):
    # Normalized the log price
    sampleCryptoLogPrice = np.log(sampleCryptoPrice)
    sampleCryptoNormalizedLogPrice = (sampleCryptoLogPrice - sampleCryptoLogPrice.mean())/sampleCryptoLogPrice.std()

    # SSD calculation for each pair
    SSDResults = []
    for i in range(len(sampleCryptoNormalizedLogPrice.columns)):
        for j in range(i+1, len(sampleCryptoNormalizedLogPrice.columns)):
            SSD = ((sampleCryptoNormalizedLogPrice[sampleCryptoNormalizedLogPrice.columns[i]] - sampleCryptoNormalizedLogPrice[sampleCryptoNormalizedLogPrice.columns[j]]) ** 2).sum()
            SSDResults.append([sampleCryptoNormalizedLogPrice.columns[i], sampleCryptoNormalizedLogPrice.columns[j], SSD])
    
    # Return the SSD results
    SSDResults = pd.DataFrame(SSDResults, columns=['Crypto 1', 'Crypto 2', 'SSD'])
    SSDResults = SSDResults.sort_values('SSD', ascending=True)
    SSDResults = SSDResults.reindex()
    
    return SSDResults

### Trading Period

In [7]:
def distanceMethodTrading(cryptoPriceDf, sampleCryptoPrice, SSDResults, cutoffDate, forward, noTradingPairs, spreadThreshold, closeThreshold, constantModel=True):
    
    # initialize the records dataframe
    TransactionRecords = pd.DataFrame(columns=['Date', 'Crypto', 'Long/Short', 'Price', "Open/Close", "Transaction pair", "Round Trip No.", "Pair No."])
    SpreadRecords = pd.DataFrame()
    PairNo = 0

    # get trading crpyto price
    cutoffRowIdx = cryptoPriceDf.index.get_loc(cutoffDate)
    if cutoffRowIdx + forward > len(cryptoPriceDf):
        # if there is not enough got the whole forward period, just get all the availiable data
        tradingCryptoPrice = cryptoPriceDf.iloc[cutoffRowIdx:]
    else:
        tradingCryptoPrice = cryptoPriceDf.iloc[cutoffRowIdx:cutoffRowIdx+forward]

    logSampleCrpytoPrice = np.log(sampleCryptoPrice)
    logTradingCryptoPrice = np.log(tradingCryptoPrice)

    for i in range(noTradingPairs):
        ############ Trading Signal Calculation ############

        # Parameter
        crypto1 = SSDResults.loc[i, 'Crypto 1']
        crypto2 = SSDResults.loc[i, 'Crypto 2']

        # Calculate normalized spread
        if constantModel:
            # constant model parameter
            crypto1Mean = logSampleCrpytoPrice.mean()[crypto1]
            crypto2Mean = logSampleCrpytoPrice.mean()[crypto2]
            crypto1SD = logSampleCrpytoPrice.std()[crypto1]
            crypto2SD = logSampleCrpytoPrice.std()[crypto2]

            # calculate spread
            normalizedCrypto1LogPrice = (logTradingCryptoPrice[crypto1] - crypto1Mean)/crypto1SD
            normalizedCrypto2LogPrice = (logTradingCryptoPrice[crypto2] - crypto2Mean)/crypto2SD
            Spread = normalizedCrypto1LogPrice - normalizedCrypto2LogPrice

        else:
            # Expending window model
            logCrypto1Price = pd.concat([logSampleCrpytoPrice[crypto1],logTradingCryptoPrice[crypto1]], axis=0)
            logCrypto2Price = pd.concat([logSampleCrpytoPrice[crypto2],logTradingCryptoPrice[crypto2]], axis=0)
            crypto1Mean = logCrypto1Price.expanding(min_periods=1).mean() # or df.rolling(window=len(df), min_periods=1).mean()
            crypto2Mean = logCrypto2Price.expanding(min_periods=1).mean()
            crypto1SD = logCrypto1Price.expanding(min_periods=1).std()
            crypto2SD = logCrypto2Price.expanding(min_periods=1).std()

            # calculate spread
            normalizedCrypto1LogPrice = (logCrypto1Price- crypto1Mean)/crypto1SD
            normalizedCrypto2LogPrice = (logCrypto2Price - crypto2Mean)/crypto2SD
            Spread = normalizedCrypto1LogPrice - normalizedCrypto2LogPrice
            Spread = Spread.loc[cutoffDate:]

        ############ Trading Execution ############
        # check if there is any trading opportunity
        SpreadWithoutLastDay = Spread.iloc[:-1]
        if len(SpreadWithoutLastDay[(SpreadWithoutLastDay >= spreadThreshold) | (SpreadWithoutLastDay <= -spreadThreshold)]) > 0:
            
            # save the spread records
            Spread.name = crypto1 + " " + crypto2
            SpreadRecords = SpreadRecords.merge(Spread, how='outer', left_index=True, right_index=True)

            # initialize before the transaction
            PairNo += 1
            Spread.name = 'spread'
            Spread = Spread.to_frame()
            Opened = False
            long = None
            RoundTripNo = 1

            for date in Spread.index:

                # When the trading date is not the last day
                if date != Spread.index[-1]:
                    # If there is an open position before that date
                    if Opened:
                        # Close postion if the spread cross closeThreshold
                        if not long and Spread.loc[date, 'spread'] <= closeThreshold:
                            TransactionRecords.loc[len(TransactionRecords)] = [date, crypto1, "Long",  tradingCryptoPrice.loc[date, crypto1], "Close", crypto2, RoundTripNo, PairNo]
                            TransactionRecords.loc[len(TransactionRecords)] = [date, crypto2, "Short",  tradingCryptoPrice.loc[date, crypto2], "Close", crypto1, RoundTripNo, PairNo]
                            RoundTripNo += 1
                            long = None
                            Opened = False
                            
                        elif long and Spread.loc[date, 'spread'] >= -closeThreshold:
                            TransactionRecords.loc[len(TransactionRecords)] = [date, crypto1, "Short",  tradingCryptoPrice.loc[date, crypto1], "Close", crypto2, RoundTripNo, PairNo]
                            TransactionRecords.loc[len(TransactionRecords)] = [date, crypto2, "Long",  tradingCryptoPrice.loc[date, crypto2], "Close", crypto1, RoundTripNo, PairNo]
                            RoundTripNo += 1
                            long = None
                            Opened = False
                            
                    
                    # Check again if there is any position, if no and fulfil the criteria, then open position 
                    if not Opened:
                        # short crypto 1 and long crypto 2 if spread >= spreadThreshold
                        if Spread.loc[date, 'spread'] >= spreadThreshold:
                            TransactionRecords.loc[len(TransactionRecords)] = [date, crypto1, "Short",  tradingCryptoPrice.loc[date, crypto1], "Open", crypto2, RoundTripNo, PairNo]
                            TransactionRecords.loc[len(TransactionRecords)] = [date, crypto2, "Long",  tradingCryptoPrice.loc[date, crypto2], "Open", crypto1, RoundTripNo, PairNo]
                            long = False
                            Opened = True
                            
                        # long crypto 1 and short crypto 2 if spread <= -spreadThreshold
                        elif Spread.loc[date, 'spread'] <= -spreadThreshold:
                            TransactionRecords.loc[len(TransactionRecords)] = [date, crypto1, "Long",  tradingCryptoPrice.loc[date, crypto1], "Open", crypto2, RoundTripNo, PairNo]
                            TransactionRecords.loc[len(TransactionRecords)] = [date, crypto2, "Short",  tradingCryptoPrice.loc[date, crypto2], "Open", crypto1, RoundTripNo, PairNo]
                            long = True
                            Opened = True
                            

                # For last day closing position
                else:
                    if Opened:
                        if not long:
                            TransactionRecords.loc[len(TransactionRecords)] = [date, crypto1, "Long",  tradingCryptoPrice.loc[date, crypto1], "Close", crypto2, RoundTripNo, PairNo]
                            TransactionRecords.loc[len(TransactionRecords)] = [date, crypto2, "Short",  tradingCryptoPrice.loc[date, crypto2], "Close", crypto1, RoundTripNo, PairNo]
                            long = None
                            Opened = False
                            
                        else:
                            TransactionRecords.loc[len(TransactionRecords)] = [date, crypto1, "Short",  tradingCryptoPrice.loc[date, crypto1], "Close", crypto2, RoundTripNo, PairNo]
                            TransactionRecords.loc[len(TransactionRecords)] = [date, crypto2, "Long",  tradingCryptoPrice.loc[date, crypto2], "Close", crypto1, RoundTripNo, PairNo]
                            long = None
                            Opened = False
                            
    return (TransactionRecords, SpreadRecords)

### Rolling Window

In [8]:
# General parameters
startDate = '2019-01-01'
endDate = '2023-07-01'
lookback =365
forward = 60

# Formation period parameters
cutoffRank = 100

# Trading period parameters
noTradingPairs = 100
spreadThreshold = 2.5
closeThreshold = 0 # same sign as spreadThreshold

In [9]:
parameters = [[3, 0],[2, 0], [1.5, 0], [1, 0], [3, 1], [2.5, 1], [2, 1], [1.5, 1]]

In [10]:
for param in parameters:

    # get parameters
    spreadThreshold = param[0]
    closeThreshold = param[1]

    TransactionRecords = pd.DataFrame(columns=['Date', 'Crypto', 'Long/Short', 'Price', "Open/Close", "Transaction pair",  "Period No.", "Pair No.", "Round Trip No.", "Quantity"])
    SpreadRecords = pd.DataFrame()

    period = 1
    for cutoffDate in pd.date_range(startDate, endDate, freq='2MS'):
        sampleCrypto = marketCapCryptoSelection(cryptoMarketCapRankDf, cutoffDate, cutoffRank)
        sampleCryptoPrice = cryptoPriceCleaning(cryptoPriceDf, sampleCrypto, cutoffDate, lookback)
        SSDResults = distanceMethodSelection(sampleCryptoPrice)
        Transaction, Spread = distanceMethodTrading(cryptoPriceDf, sampleCryptoPrice, SSDResults, cutoffDate, forward, noTradingPairs, spreadThreshold, closeThreshold, False)
        Transaction['Period No.'] = period
        TransactionRecords = pd.concat([TransactionRecords, Transaction], ignore_index=True)
        SpreadRecords = pd.concat([SpreadRecords, Spread])
        period += 1

    TransactionRecords['Quantity'] = 1
    TransactionRecords.to_csv('Transaction/open ' + str(spreadThreshold) + " close " + str(closeThreshold) + '/Transactions_distance.csv')
    SpreadRecords.to_csv('Transaction/open ' + str(spreadThreshold) + " close " + str(closeThreshold) + '/SpreadRecord_distance.csv')

Remaining number of crpyto:  45
Remaining number of crpyto:  51
Remaining number of crpyto:  55
Remaining number of crpyto:  49
Remaining number of crpyto:  47
Remaining number of crpyto:  48
Remaining number of crpyto:  46
Remaining number of crpyto:  47
Remaining number of crpyto:  49
Remaining number of crpyto:  52
Remaining number of crpyto:  52
Remaining number of crpyto:  57
Remaining number of crpyto:  60
Remaining number of crpyto:  58
Remaining number of crpyto:  62
Remaining number of crpyto:  63
Remaining number of crpyto:  66
Remaining number of crpyto:  67
Remaining number of crpyto:  65
Remaining number of crpyto:  65
Remaining number of crpyto:  68
Remaining number of crpyto:  77
Remaining number of crpyto:  84
Remaining number of crpyto:  78
Remaining number of crpyto:  82
Remaining number of crpyto:  82
Remaining number of crpyto:  88
Remaining number of crpyto:  88
Remaining number of crpyto:  45
Remaining number of crpyto:  51
Remaining number of crpyto:  55
Remainin

### Transform transaction records to Result

In [10]:
def transformTransactionRecords(TransactionRecords):
    result = pd.DataFrame(columns=['Period No.', 'Pair No.', 'Round Trip No.', 'Start Date', 'End Date', 'crypto 1', 'crypto 2', 'crypto 1 return', 'crypto 2 return', 'crypto 1 Long/Short', 'crypto 2 Long/Short', 'Quantity'])
    # loop each period
    for k in range(1, TransactionRecords['Period No.'].max() + 1):
        period =  TransactionRecords[TransactionRecords['Period No.'] == k]
        
        # if there is no trade in that period
        if len(period) == 0:
            continue
        
        # loop each pair of transactions
        for i in range(1, period['Pair No.'].max() + 1):
            pair = period[period['Pair No.'] == i]
            # loop each Round Trip in pair
            for j in range(1, pair['Round Trip No.'].max() + 1):
                roundTrip = pair[pair['Round Trip No.'] == j]

                returnResult = dict()

                # loop each crypto in the round trip
                for crypto in set(roundTrip['Crypto']):
                    
                    # prepare the specific round trip transaction record
                    record = roundTrip[roundTrip['Crypto'] == crypto]
                    record = record.reset_index(drop=True)

                    # Calculate the return of the specific round trip
                    returns = record['Price'][1] / record['Price'][0] - 1
                    if record['Long/Short'][0] == 'Short':
                        returns = -returns 
                    
                    # Insert Record
                    if len(returnResult) == 0:
                        returnResult['Period No.'] = k
                        returnResult['Pair No.'] = i
                        returnResult['Round Trip No.'] = j
                        returnResult['Start Date'] = record['Date'][0]
                        returnResult['End Date'] = record['Date'][1]
                    
                    if record['Long/Short'][0] == 'Long':
                        returnResult['crypto 1'] = crypto
                        returnResult['crypto 1 return'] = returns
                        returnResult['crypto 1 Long/Short'] = 1
                    else:
                        returnResult['crypto 2'] = crypto
                        returnResult['crypto 2 return'] = returns
                        returnResult['crypto 2 Long/Short'] = -1
                        returnResult['Quantity'] = 1

                result.loc[len(result)] = returnResult  
    return result


In [41]:
# TransactionRecords_distance = pd.read_csv('Transaction/Transactions_distance.csv', index_col=0)
# TransactionRecords_distance['Date'] = pd.to_datetime(TransactionRecords_distance['Date'])
# result_distance = transformTransactionRecords(TransactionRecords_distance)

### Daily Return

In [42]:
# def getcumReturns(cryptoPriceDf, result):
#     cumReturn = pd.DataFrame()
#     # loop through each row
#     for idx, row in result.iterrows():
#         crpyto1StartPrice = cryptoPriceDf.loc[row['Start Date'], row['long crypto']]
#         crpyto2StartPrice = cryptoPriceDf.loc[row['Start Date'], row['short crypto']]
#         crpyto1cumReturn = cryptoPriceDf.loc[row['Start Date']:row['End Date'], row['long crypto']] / crpyto1StartPrice - 1
#         crpyto2cumReturn = cryptoPriceDf.loc[row['Start Date']:row['End Date'], row['short crypto']] / crpyto2StartPrice - 1
#         cumReturnSeries = crpyto1cumReturn - crpyto2cumReturn
#         cumReturnSeries.name = row['long crypto'] + " " + row['short crypto'] + " " + str(row['Period No.']) + + " " + str(row['Round Trip No.'])
#         cumReturn = cumReturn.merge(cumReturnSeries, left_index=True, right_index=True, how="outer")
#     return cumReturn

In [43]:
# cumReturns = getcumReturns(cryptoPriceDf, result)
# dailyReturns = np.exp(np.log(cumReturns + 1).diff()) - 1 # OR (cumReturns + 1) / (cumReturns.shift(1) + 1) - 1
# averageDailyReturn = dailyReturns.mean(axis=1)
# averageDailyReturn = averageDailyReturn.fillna(0)
# averageCumReturn = np.cumprod(1 +averageDailyReturn) - 1

In [12]:
def getcumReturns(cryptoPriceDf, result):
    cumReturn = pd.DataFrame()
    # loop through each row
    for idx, row in result.iterrows():
        crpyto1StartPrice = cryptoPriceDf.loc[row['Start Date'], row['crypto 1']]
        crpyto2StartPrice = cryptoPriceDf.loc[row['Start Date'], row['crypto 2']]
        crpyto1cumPnL = (cryptoPriceDf.loc[row['Start Date']:row['End Date'], row['crypto 1']] - crpyto1StartPrice) * row['crypto 1 Long/Short'] 
        crpyto2cumPnL = (cryptoPriceDf.loc[row['Start Date']:row['End Date'], row['crypto 2']] - crpyto2StartPrice) * row['crypto 2 Long/Short'] * row['Quantity']
        cumReturnSeries = (crpyto1cumPnL + crpyto2cumPnL) / (crpyto1StartPrice * 0.5 + crpyto2StartPrice * row['Quantity'] * 0.5) 
        cumReturnSeries.name = row['crypto 1'] + " " + row['crypto 2'] + " " + str(row['Period No.']) + " " + str(row['Round Trip No.'])
        cumReturn = cumReturn.merge(cumReturnSeries, left_index=True, right_index=True, how="outer")
    return cumReturn

In [45]:
# # Remark: return can be more than -100% for the short selling position
# cumReturns = getcumReturns(cryptoPriceDf, result_distance)
# result_distance['Total Return'] = cumReturns.ffill(axis=0).iloc[-1].reset_index(drop=True)

In [13]:
for param in parameters:

    # get parameters
    spreadThreshold = param[0]
    closeThreshold = param[1]
    
    TransactionRecords_distance = pd.read_csv('Transaction/open ' + str(spreadThreshold) + " close " + str(closeThreshold) + '/Transactions_distance.csv', index_col=0)
    TransactionRecords_distance['Date'] = pd.to_datetime(TransactionRecords_distance['Date'])
    result_distance = transformTransactionRecords(TransactionRecords_distance)
    cumReturns = getcumReturns(cryptoPriceDf, result_distance)
    result_distance['Total Return'] = cumReturns.ffill(axis=0).iloc[-1].reset_index(drop=True)

    result_distance.to_csv('Result/open ' + str(spreadThreshold) + " close " + str(closeThreshold) + '/Result_distance.csv')